In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.gold;


In [0]:
from pyspark.sql.functions import sum, avg, col

# 1. Read Silver (trusted data)
silver_df = spark.read.table("workspace.silver.population_clean")

# 2. Business aggregation (Gold logic)
gold_df = (
    silver_df
    .groupBy("state_code")
    .agg(
        sum("population_estimate_2014").alias("total_population_2014"),
        avg("median_sales_price_2015").alias("avg_median_sales_price_2015")
    )
)

# 3. Write Gold Unity Catalog table
gold_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.gold.population_summary")

display(gold_df)


In [0]:
%sql
SELECT *
FROM workspace.gold.population_summary
ORDER BY total_population_2014 DESC;